In [76]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [77]:
tf.reset_default_graph()

In [78]:
sentences = ['ich mochte ein bier P', 'S i want a beer', 'i want a beer E']

In [79]:
word_list = " ".join(sentences).split()
print(word_list)

['ich', 'mochte', 'ein', 'bier', 'P', 'S', 'i', 'want', 'a', 'beer', 'i', 'want', 'a', 'beer', 'E']


In [80]:
word_list = list(set(word_list))
print(word_list)

['E', 'S', 'want', 'a', 'i', 'bier', 'P', 'ein', 'beer', 'mochte', 'ich']


In [81]:
word_dict = {w: i for i, w in enumerate(word_list)}
print(word_dict)

{'E': 0, 'S': 1, 'want': 2, 'a': 3, 'i': 4, 'bier': 5, 'P': 6, 'ein': 7, 'beer': 8, 'mochte': 9, 'ich': 10}


In [82]:
number_dict = {i: w for i, w in enumerate(word_list)}
print(number_dict)

{0: 'E', 1: 'S', 2: 'want', 3: 'a', 4: 'i', 5: 'bier', 6: 'P', 7: 'ein', 8: 'beer', 9: 'mochte', 10: 'ich'}


In [83]:
n_class = len(word_dict)  # vocab list
print(n_class)

11


In [84]:
n_step = 5
n_hidden = 128

In [85]:
def make_batch(sentences):
    input_batch = [np.eye(n_class)[[word_dict[n] for n in sentences[0].split()]]]
    output_batch = [np.eye(n_class)[[word_dict[n] for n in sentences[1].split()]]]
    target_batch = [[word_dict[n] for n in sentences[2].split()]]
    return input_batch, output_batch, target_batch


In [86]:
enc_inputs = tf.placeholder(tf.float32, [None, None, n_class])
dec_inputs = tf.placeholder(tf.float32, [None, None, n_class])
targets = tf.placeholder(tf.int64, [1, n_step])

In [87]:
attn = tf.Variable(tf.random_normal([n_hidden, n_hidden]))
out = tf.Variable(tf.random_normal([n_hidden *2, n_class]))

In [88]:
def get_att_score(dec_output, enc_output):
    score = tf.squeeze(tf.matmul(enc_output, attn), 0)
    dec_output = tf.squeeze(dec_output, [0, 1])
    return tf.tensordot(dec_output, score, 1)

In [89]:
def get_att_weight(dec_output, enc_outputs):
    attn_scores = []
    enc_outputs = tf.transpose(enc_outputs, [1, 0, 2])
    for i in range(n_step):
        attn_scores.append(get_att_score(dec_output, enc_outputs[i]))
    return tf.reshape(tf.nn.softmax(attn_scores), [1, 1, -1])

In [75]:
model = []
Attention = []
with tf.variable_scope('encode'): 
    enc_cell =tf.keras.layers.SimpleRNNCell(n_hidden)
    rnn = tf.keras.layers.RNN(enc_cell)
    output = rnn(enc_inputs)
    
    rnn = tf.keras.layers.RNN(
        enc_cell,
        return_sequences=True,
        return_state=True)
    
    enc_outputs, enc_hidden = rnn(enc_cell)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


ValueError: as_list() is not defined on an unknown TensorShape.

In [81]:
with tf.variable_scope('decode'):
    dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)  

In [82]:
    inputs = tf.transpose(dec_inputs, [1, 0, 2])
    hidden = enc_hidden
    for i in range(n_step):
        dec_output, hidden = tf.nn.dynamic_rnn(dec_cell, tf.expand_dims(inputs[i], 1),
                                              initial_state=hidden, dtype=tf.float32, time_major=True)
        attn_weights = get_att_weight(dec_output, enc_outputs)
        Attention.append(tf.squeeze(attn_weights))

In [83]:
        context = tf.matmul(attn_weights, enc_outputs)
        dec_output = tf.squeeze(dec_output, 0)
        context = tf.squeeze(context, 1)